# **Setup**
 
Reset the Python environment to clear it of any previously loaded variables, functions, or libraries. Then, import the libraries needed to complete the code Professor Melnikov presented in the video.

In [ ]:
%reset -f
from IPython.core.interactiveshell import InteractiveShell as IS
IS.ast_node_interactivity = "all"    # allows multiple outputs from a cell
import numpy as np, pandas as pd, seaborn as sns, matplotlib.pyplot as plt, scipy, json
from sentence_transformers import SentenceTransformer
from sklearn.cluster import KMeans              # Use ?KMeans to view help documents
from sklearn.metrics import silhouette_score    # Use ?silhouette_score to view help documents
from sklearn.decomposition import PCA   # PCA uses SVD to reduce dimensionality of the feature space
import plotly.graph_objects as go           # import graph object from plotly library

pd.set_option('max_rows', 5, 'max_columns', 20, 'max_colwidth', 100, 'precision', 2)


## **Load TMDB and SBERT**

Recall that the Movie Database ([TMDB](https://www.themoviedb.org/)) file (`movies.zip`) contains 4803 movies (rows) and 19 features (columns), which can be textual and numeric. It is loaded below into `df` dataframe with title as index labels.

<span style="color:black"> The code below also loads the Sentence Transformer ([SBERT](https://www.sbert.net/) ), which converts any length text to a fixed size numeric vector of 768 dimensions. Unlike in the video, a much smaller pre-trained language model, `'paraphrase-albert-small-v2'`, is loaded here for the sake of speed. It is 50 MB, which is about seven times smaller than the one in the video. Note that in a production environment, the speed needs to be balanced with the embedding quality, which in itself has to be mathematically sound.

In [ ]:
%time SBERT = SentenceTransformer('paraphrase-albert-small-v2')   # load a pre-trained model, 250MB
df0 = pd.read_csv('movies.zip').fillna('')  # load TMDB file
df0.index = df0.title        # set index labels for visual convenience only

## **Filter Movies and Build a More Comprehensive Descriptions**

<span style="color:black"> In the video, 1154 `Action` genre movies were used. Here, to speed up the encoding, the cell below further selects movies that are both `Action` and `Family` genre movies, leaving only 62 films for processing.
    
<span style="color:black"> Also, the code below concatenates title, tagline, and overview of each movie into a `Desc` field, which will later be encoded. The hypothesis is that more diverse and descriptive text gives better results. However, the embeddings are unlikely to improve if you concatenate the `overview` attribute 100 times. The JSON parsing seen in the video is left out to shorten the setup.

In [ ]:
dfS = df0[df0.genres.str.contains('Action') & df0.genres.str.contains('Family')]   # sample of movies
df = (dfS.title +'. '+ dfS.tagline +'. '+ dfS.overview).to_frame().rename(columns={0:'Desc'})
df

## **Encode Movies with SBERT Language Model**

The next cell passes a list of all movie descriptions through the embedding model and packages the results as a numeric dataframe with all coefficients packed in 768 columns. Each row is a numeric vector for the given movie with a title in the row index.

In [ ]:
%time mEmb = SBERT.encode(df.Desc) # Encoding textual descriptions with numeric vectors
dfEmb = pd.DataFrame(mEmb, index=df.index)
dfEmb

<hr style="border-top: 2px solid #606366; background: transparent;">

# **Review**

Review the code Professor Melnikov used to apply *k*-means clustering to movie vectors.

## **Cluster with K-Means**

<span style="color:black">The next cell applies [*k*-means clustering algorithm](https://scikit-learn.org/stable/modules/clustering.html#k-means) to search for three clusters. The algorithm initializes with three random points, $a_0,b_0,c_0$, and all points are assigned to the closest point. Then, centroids $a_1,b_1,c_1$ of each cluster are computed. Importantly, this clustering is sensitive to initialization and different clusters can be formed with different starting points. 
    
<span style="color:black">One way to find the "most representative" clusters is to run this algorithm, multiple times and at each iteration new $a_i,b_i,c_i$ are computed until no more re-assignment occurs. Then, you pick the cluster set that appears most frequently. Another approach is to pick a cluster set that results in the highest clustering metric, such as silhouette score.

In [ ]:
km = KMeans(n_clusters=3, random_state=0, n_init=30, max_iter=1000).fit(dfEmb)
print(f'Number of executed iterations: {km.n_iter_}')  # completed recomputations of centroids
print(km)  # print the model declaration

As with hierarchical clustering, you can extract the cluster assignments via [`labels_`](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html) attribute. This is done below to save each movie's assignment to a cluster. The algorithm has identified three clusters, decreasing in size (number of contained movies).

In [ ]:
df['kmeans_cluster'] = km.labels_                 # assign identified labels to action movies
df['kmeans_cluster'].value_counts().to_frame().T  # compute number of movies in each identified cluster

Use an attribute [`cluster_centers_`](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html) to retrieve the identified centroids, which are 768-dimensional just like all other movie vectors.

In [ ]:
pd.DataFrame(km.cluster_centers_).iloc[:,-20:].round(2)   # show centroids

<span style="color:black"> To measure the quality of the clustering, you can compute the [`silhouette_score()`](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.silhouette_score.html), which is done in the cell below. This metric approaches 1 for clusters which are tight and far from neighboring clusters. It returns values closer to -1 for clusters which are relatively dispersed and overlaying. With `KMeans`, overlaying in 768D space will not occur since all same-cluster points form a disk around their centroid. However, cluster intersection can appear in a plot when 2D approximations replace 768D points.
    
As usual, you should try to improve this metric towards its perfect value. Typically, the improvements are exponentially harder to make. You should continue seeking such improvements while it's still feasible time-wise and cost-wise. Each corpus will have its own starting point, which can be 0.1 for some and 0.9 for the others. 

In [ ]:
silhouette_score(dfEmb, km.labels_)   # unsupervised clustering metric

Next, RGB color strings are generated for the clusters using [`sns.color_palette`](https://seaborn.pydata.org/generated/seaborn.color_palette.html) function.

In [ ]:
LsPalette = [f'rgb({c[0]},{c[1]},{c[2]})' for c in sns.color_palette('bright', km.n_clusters)]  # strings of RGB color values
vColors = np.array(LsPalette)[km.labels_]   # vector of colors (as RGB string) for each point 
vColors[:2]

<span style="color:black">Then, we approximate each movie vector with a 2D counterpart so that movie points could be visualized in 2D plane using Principal Component Analysis ([PCA](https://scikit-learn.org/stable/modules/decomposition.html#pca)), which is easily applied to the matrix of movie vectors.

In [ ]:
mPC12 = PCA(n_components=2).fit_transform(dfEmb)   # project 768-dim vectors to 2D space for plotting
dfPC12 = pd.DataFrame(mPC12, columns=['x','y'], index=df.index)
dfPC12                                  # contains new (x,y) coordinates and cluster label

<span style="color:black"> In the following cell, all three clusters are plotted in color. An expert should evaluate whether the given clusters are sensible for the given input descriptions. Alternatively, one could compute the silhouette score to automatically identify optimal `KMeans` parameters and input documents. It's important to note that visual evaluation of clusters is often helpful to evaluate and optimize clustering using domain expertise.

In [ ]:
LsG = dfS.genres.apply(lambda s: ', '.join([d['name'] for d in json.loads(s)])).tolist() # extract genres from JSONs
sMovieGenres = [t + '; ' + g for t,g in zip(dfS1.index, LsG)]              # point labels with title+genre
DMarkers = dict(size=3, line=dict(width=1, color=vColors), color=vColors)  # plot marker definition (as a dictionary)
goMargin = go.layout.Margin(l=0, r=0, b=0, t=0)                            # plot margin definition

goS = go.Scatter(x=dfPC12.x, y=dfPC12.y, mode='markers', marker=DMarkers, text=sMovieGenres, name='movies'); # plot definition
goLayout = go.Layout(hovermode='closest', margin=goMargin, width=800, 
                   height=500, xaxis={'title':'PC1'}, yaxis={'title':'PC2'}); # plot layout definition

print('Action movie clusters')
fig = go.Figure(layout=goLayout)  # prepare a figure with specified layout
fig.add_trace(goS)                # plot movie points

## **Hyperparameter Grid Search**

<span style="color:black">Automated selection of the (hyper-) parameters can be done by trying all desirable parameter values. However, trying all possible values is not feasible and is unnecessary. Automated selection can still be utilizied combined with human insight. For example, an NLP engineer should develop an intuition on the approximate number of desired clusters, $k$. Then, you can try a slightly larger range for $k$, which is computationally more effective.

Below there is demonstration of an automatic selection of the parameter $k$. Having computed each metric for $k\in\{2,10\}$, you can mechanically pick the smallest $k$ with the largest silhouette score (recall that there might be more than one such optimal $k$ value).

In [ ]:
K = range(2, 11)  # different numbers of clusters
def KM(k=2) -> (float, float, str):
    km = KMeans(n_clusters=k, random_state=0, n_init=20).fit(dfEmb)  # create and fit KMeans to movie vectors
    return k, silhouette_score(dfEmb, km.labels_), km                           # return k, score, model

%time vModels = np.array([KM(k) for k in K])

sTtl = 'Silhouette Scores for KMean Clustering Models'
pd.DataFrame(vModels[:,0:2], columns=['k', 'SIL']).set_index('k').plot(grid=True, title=sTtl, figsize=[15,3]);

<hr style="border-top: 2px solid #606366; background: transparent;">

# **Optional Practice**

Now you will practice building *k*-means models with different hyperparameters and choosing the set of these parameters that yield the best clustering perfmance in terms of a silhuette score.

As you work through these tasks, check your answers by running your code in the *#check solution here* cell, to see if you’ve gotten the correct result. If you get stuck on a task, click the See **solution** drop-down to view the answer.

## **Task 1**

Compute the silhouette scores for *k*-means clustering with 

1. 2 and 3 clusters
1. 0 and 1 random states
1. 10 and 20 initializations.
1. `'k-means++'` and  `'random'` initializations (`init` argument to [`KMeans()`](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html))
1. `'full'` and `'elkan'` algorithms (`algorithm` argument to [`KMeans()`](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html)

Overall, you will have $2^5=32$ combinations of parameters, i.e., 32 calls to `KMeans()` algorithms.

Which of these hyper-parameters do you favor and why?

 <b>Hint:</b> You can brute-force each fitting and then visually evaluate all 32 silhouette scores. Alternatively, you can create a list of all 32 instantiated `KMeans` objects and use list comprehension to fit each object on <code>dfEmb</code> and use its labels to compute 32 silhouette scores. The largest scores are preferred with fewer number of clusters.

In [ ]:
# check solution here

<font color=#606366>
    <details><summary><font color=#B31B1B>▶ </font>See <b>solution</b>.</summary>
<pre class="ec">
models = [
KMeans(2, random_state=0, n_init=20, init='k-means++', algorithm='full'),
KMeans(2, random_state=0, n_init=20, init='k-means++', algorithm='elkan'),
KMeans(2, random_state=0, n_init=20, init='random',    algorithm='full'),
KMeans(2, random_state=0, n_init=20, init='random',    algorithm='elkan'),
KMeans(2, random_state=0, n_init=10, init='k-means++', algorithm='full'),
KMeans(2, random_state=0, n_init=10, init='k-means++', algorithm='elkan'),
KMeans(2, random_state=0, n_init=10, init='random',    algorithm='full'),
KMeans(2, random_state=0, n_init=10, init='random',    algorithm='elkan'),
    
KMeans(2, random_state=1, n_init=20, init='k-means++', algorithm='full'),
KMeans(2, random_state=1, n_init=20, init='k-means++', algorithm='elkan'),
KMeans(2, random_state=1, n_init=20, init='random',    algorithm='full'),
KMeans(2, random_state=1, n_init=20, init='random',    algorithm='elkan'),
KMeans(2, random_state=1, n_init=10, init='k-means++', algorithm='full'),
KMeans(2, random_state=1, n_init=10, init='k-means++', algorithm='elkan'),
KMeans(2, random_state=1, n_init=10, init='random',    algorithm='full'),
KMeans(2, random_state=1, n_init=10, init='random',    algorithm='elkan'),

KMeans(3, random_state=0, n_init=20, init='k-means++', algorithm='full'),
KMeans(3, random_state=0, n_init=20, init='k-means++', algorithm='elkan'),
KMeans(3, random_state=0, n_init=20, init='random',    algorithm='full'),
KMeans(3, random_state=0, n_init=20, init='random',    algorithm='elkan'),
KMeans(3, random_state=0, n_init=10, init='k-means++', algorithm='full'),
KMeans(3, random_state=0, n_init=10, init='k-means++', algorithm='elkan'),
KMeans(3, random_state=0, n_init=10, init='random',    algorithm='full'),
KMeans(3, random_state=0, n_init=10, init='random',    algorithm='elkan'),
    
KMeans(3, random_state=1, n_init=20, init='k-means++', algorithm='full'),
KMeans(3, random_state=1, n_init=20, init='k-means++', algorithm='elkan'),
KMeans(3, random_state=1, n_init=20, init='random',    algorithm='full'),
KMeans(3, random_state=1, n_init=20, init='random',    algorithm='elkan'),
KMeans(3, random_state=1, n_init=10, init='k-means++', algorithm='full'),
KMeans(3, random_state=1, n_init=10, init='k-means++', algorithm='elkan'),
KMeans(3, random_state=1, n_init=10, init='random',    algorithm='full'),
KMeans(3, random_state=1, n_init=10, init='random',    algorithm='elkan'),
]
sils = ([(silhouette_score(dfEmb, km.fit(dfEmb).labels_), km) for km in models])
pd.set_option('precision', 5)
pd.DataFrame(sils, columns=['SIL', 'model']).sort_values('SIL', ascending=False)
            </pre>The random state should not be considered in this search for the best hyperparameters. Otherwise, we prefer the hyperparameters with the highest silhouette score. If more than one such set of hyperparameters exist, then we favor a simpler model among these, i.e., the model with fewer clusters and "more robust" (i.e. less sensitive to, say, random state) initialization and algorithm.
</details> 
</font>

<hr>

## Task 2

Modify UDF `KM()` to build a new UDF `KM1(k=2, init='k-means++') -> (float, float, str)`, which also takes `init` parameter (with values `k-means++` or `random`) and returns a tuple with values `k`, `init` and `SIL`. Then find the optimal (hyper-) parameter values, i.e., those that yield the highest silhouette score. (Plotting is not necessary, but helps visualize the performance of the increasing $k$ per each `init` value).

<b>Hint:</b> After you make changes to <code>KM()</code> UDF, you can call it in two separate list comprehensions with different <code>init</code> parameters and concatenate the outputs.

In [ ]:
# check solution here

<font color=#606366>
    <details><summary><font color=#B31B1B>▶ </font>See <b>solution</b>.</summary>
        <pre class="ec">
def KM1(k=2, init='k-means++') -> (float, float, str):
    km = KMeans(n_clusters=k, random_state=0, n_init=20, init=init).fit(dfEmb) 
    return k, init, silhouette_score(dfEmb, km.labels_)    # return k, init, SIL score

K = range(2, 11)  # different numbers of clusters
L_KM = [KM1(k, 'k-means++') for k in K] + [KM1(k, 'random') for k in K]
dfKM = pd.DataFrame(L_KM, columns=['k','init','SIL'])
print(dfKM.sort_values('SIL', ascending=False).iloc[0,:])

sTtl = 'Silhouette score for different values of k in KMeans clustering'
dfKM.set_index('k').groupby('init')['SIL'].plot(legend=True,grid=True,title=sTtl,figsize=[15,3]);
            </pre>Note that the highest SIL score is with two clusters, but this is only among the cluster numbers varying from 2 to 10. Perhaps, some higher *k* than 10 yields a higher SIL score. The only way to find out is to evaluate higher values of *k*. Since this can be expensive, you should evaluate just a few *k* values. However, these do not need to be consequent. You could try *k*=2,20,200 or any other combination. Then you could focus on the narrower range of *k* values. Notably, SIL=0.03(3) tells us that clustering is relatively poor and has room for improvement.
        </details>
    </details> 
</font>

<hr>

In [ ]:
def KM1(k=2, init='k-means++') -> (float, float, str):
    km = KMeans(n_clusters=k, random_state=0, n_init=20, init=init).fit(dfEmb) 
    return k, init, silhouette_score(dfEmb, km.labels_)    # return k, init, SIL score

K = range(2, 11)  # dufferent numbers of clusters
L_KM = [KM1(k, 'k-means++') for k in K] + [KM1(k, 'random') for k in K]
dfKM = pd.DataFrame(L_KM, columns=['k','init','SIL'])
print(dfKM.sort_values('SIL', ascending=False).iloc[0,:])


sTtl = 'Silhouette score for different values of k in KMeans clustering'
dfKM.set_index('k').groupby('init')['SIL'].plot(legend=True,grid=True,title=sTtl,figsize=[15,3]);